<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/deep-learning-with-python-2nd/blob/master/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</td></tr></table>

In [ ]:
!pip install -q keras-tuner

     |████████████████████████████████| 135 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 43.9 MB/s 


In [ ]:
import keras_tuner as kt
from tensorflow import keras

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_val = x_train[:-10000], x_train[-10000:]
y_train, y_val = y_train[:-10000], y_train[-10000:]

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.reshape((-1, 28, 28, 1)) / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)) / 255.0
x_val = x_val.reshape((-1, 28, 28, 1)) / 255.0

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(28, 28, 1))
    filters = hp.Int("filters_1", min_value=16, max_value=64, step=16)
    x = keras.layers.Conv2D(filters, 3, activation="relu")(inputs)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    filters = hp.Int("filters_2", min_value=32, max_value=512, step=16)
    x = keras.layers.Conv2D(filters, 3, activation="relu")(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    filters = hp.Int("filters_3", min_value=32, max_value=512, step=16)
    x = keras.layers.Conv2D(filters, 3, activation="relu")(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)
    x = keras.layers.Flatten()(x)
    outputs = keras.layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    optimizer = hp.Choice("optimizer", values=["rmsprop", "adam"])
    lr = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
    if optimizer == "rmsprop":
        opt = keras.optimizers.RMSprop(learning_rate=lr)
    else:
        opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(opt, loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
model = build_model(kt.HyperParameters())
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 32)          9248  

In [ ]:
tuner = kt.RandomSearch(build_model, objective="val_accuracy", 
                        max_trials=100, seed=42)
tuner.search_space_summary()

Search space summary
Default search space size: 5
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
filters_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 16, 'sampling': None}
filters_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}
lr (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [ ]:
callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)]
tuner.search(x_train, y_train, batch_size=64, epochs=100,
             validation_data=(x_val, y_val), callbacks=callbacks, verbose=2)

Trial 100 Complete [00h 01m 56s]
val_accuracy: 0.8949000239372253

Best val_accuracy So Far: 0.9114999771118164
Total elapsed time: 02h 36m 50s


In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
filters_1: 64
filters_2: 480
filters_3: 464
optimizer: rmsprop
lr: 0.0005531280941472172
Score: 0.9114999771118164
Trial summary
Hyperparameters:
filters_1: 64
filters_2: 512
filters_3: 128
optimizer: adam
lr: 0.0002906286580403997
Score: 0.9071999788284302
Trial summary
Hyperparameters:
filters_1: 32
filters_2: 416
filters_3: 352
optimizer: rmsprop
lr: 0.00030092742988110965
Score: 0.9067000150680542
Trial summary
Hyperparameters:
filters_1: 16
filters_2: 272
filters_3: 336
optimizer: rmsprop
lr: 0.0007343120079351318
Score: 0.9061999917030334
Trial summary
Hyperparameters:
filters_1: 32
filters_2: 352
filters_3: 192
optimizer: adam
lr: 0.0010464958375249103
Score: 0.9056000113487244
Trial summary
Hyperparameters:
filters_1: 48
filters_2: 480
filters_3: 432
optimizer: adam
lr: 0.0009447404851439078
Score: 0.9050999879837036
Trial summary
Hyperparameters:
filters_1: 32
filters_2: 272
fil

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)
best_hps[0].values

{'filters_1': 64,
 'filters_2': 480,
 'filters_3': 464,
 'optimizer': 'rmsprop',
 'lr': 0.0005531280941472172}

In [ ]:
best_models = tuner.get_best_models(1)
final_model = best_models[0]

In [ ]:
final_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3429 - accuracy: 0.9049


[0.3428693115711212, 0.9049000144004822]